# 1. Use a Command Job to create a Machine Learning Model

A Command Job in Azure Machine Learning (Azure ML) is a job type that allows you to execute a Python script, shell command, or any executable in a containerized environment. It is often used to run machine learning training scripts, data preprocessing steps, or custom tasks in Azure ML.

For command job the below arguments are required
- code: Path to the directory containing the script or code files.
- command: The command to execute, often invoking the script (e.g., python train.py).
- inputs: Dictionary of inputs passed to the script (e.g., data paths, hyperparameters).
- outputs: Dictionary of outputs produced by the script (e.g., model files).
- environment: The runtime environment, including dependencies (e.g., Python packages, Docker image).
- compute: The compute target to use (e.g., a virtual machine or compute cluster in Azure ML).
- description: A brief description of the job's purpose.


In [1]:
## Create a handle to Workspace
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
subscription='73f10b45-ec04-427e-a5d6-42ed4e79c537'
resourcegroup='MLRG01'
workspace='AzureMLWS01'

ml_client=MLClient(credential=DefaultAzureCredential(),
subscription_id=subscription,
resource_group_name=resourcegroup,
workspace_name=workspace)

In [2]:
ml_client.workspace_name

'AzureMLWS01'

In [3]:
ws=ml_client.workspaces.get(workspace)
ws.name, ws.resource_group

('AzureMLWS01', 'MLRG01')

# 2. Create the job Env

In [8]:
import os
dependecies_dir='../dependencies'
os.makedirs(dependecies_dir,exist_ok=True)
#Users/mr.snath88/dependencies
## then upload the conda.yaml file

In [13]:
from azure.ai.ml.entities import Environment

custom_job_env = Environment(
    name="sklearn-env",
    description="Environment for sklearn-based training",
    conda_file="../dependencies/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest",
)

custom_job_env=ml_client.environments.create_or_update(custom_job_env)

print(f'Env: {custom_job_env.name} created and the version is {custom_job_env.version}')

Env: sklearn-env created and the version is 9


# 3. Create Training Script

In [14]:
## create directory
script_path='../src'
os.makedirs(script_path,exist_ok=True)

# 4. Configure the command

In [24]:
from azure.ai.ml import command
# Define the command job
# job = command(
#     code="./",  # Path to the directory containing the script
#     command="python training_script_with_argparse.py --data Social_Network_Ads --test_train_ratio 0.2 --estimator random_forest --registered_model_name my_registered_model",
#     environment="azureml:my-custom-environment:1",  # Predefined environment or use your own
#     compute="<your-compute-cluster>",  # Cluster for execution
#     experiment_name="train_with_argparse",
# )

###defining the arguments by dictionary

input_params = {
    "data": "Social_Network_Ads",
    "test_train_ratio": 0.2,
    "estimator": "random_forest",
    "registered_model_name": "Social_Media_Model",
}

arguments = " ".join([f"--{key} {value}" for key, value in input_params.items()])

job = command(
    code="../src/",  # Path to the directory containing the script
    command=f"python main.py {arguments}",
    environment="sklearn-env@latest",  # Predefined environment or use your own
    experiment_name="Social_Media_exp",
    display_name='Social_mdeia_Ads_classification',
)

ml_client.jobs.create_or_update(job)

Uploading src (0.01 MBs): 100%|██████████| 8234/8234 [00:00<00:00, 126008.94it/s]




Experiment,Name,Type,Status,Details Page
Social_Media_exp,salmon_panda_t77k6s5l3r,command,Starting,Link to Azure Machine Learning studio


Get the latest model

In [10]:
model_name=''
# model=list(ml_client.models.list())
for models in ml_client.models.list():
    print(models.name)

amlstudio-azureml-loan-applica
azureml_b3128233-70ba-405d-b193-77a5d1dcaa0e_output_mlflow_log_model_1858157903
azureml_9c21282c-915d-469a-ad5e-df7e8a8c0c28_output_mlflow_log_model_662056380
azureml_0b62c88f-676b-42fc-8434-c4918047c689_output_mlflow_log_model_662056380
azureml_828fa006-2fbf-435f-a134-1726fe49133b_output_mlflow_log_model_398141616
azureml_ab0b36c3-ee29-4f87-b5cb-db44d2e9c337_output_mlflow_log_model_1858157903
azureml_salmon_panda_t77k6s5l3r_output_mlflow_log_model_662056380
azureml_salmon_panda_t77k6s5l3r_output_mlflow_log_model_917862491
Social_mdedia_model


# 5. Creating the endpoints

In [23]:
import uuid
str(uuid.uuid4())[:8]

'e99b9eab'